# Virtual Concierge

## Host pre-trained endpoint for MXNET model

In this notebook we will download a pre-trained MXNET model and deploy endpoint

In [ ]:
!pip -q install mxnet==1.3.1
!pip -q install sagemaker==1.18.9.post1

In [ ]:
import platform
import mxnet as mx
import sagemaker

print('pyversion: {}, mxnet: {}, sagemaker: {}'.format(
    platform.python_version(), mx.__version__, sagemaker.__version__))

In [ ]:
# Download pre-trained model if haven't already created from previous notebook
import os

if not os.path.exists('model.tar.gz'):
    !aws s3 cp s3://deeplens-virtual-concierge-model/mobilefacenet/model.tar.gz .

### Import model into SageMaker

Open a new sagemaker session and upload the model on to the default S3 bucket. We can use the ``sagemaker.Session.upload_data`` method to do this. We need the location of where we exported the model from MXNet and where in our default bucket we want to store the model(``/model``). The default S3 bucket can be found using the ``sagemaker.Session.default_bucket`` method.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
model_data = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='virtual-concierge')

Use the ``sagemaker.mxnet.model.MXNetModel`` to import the model into SageMaker that can be deployed. We need the location of the S3 bucket where we have the model, the role for authentication and the entry_point where the model defintion is stored (``predict.py``). 

In [ ]:
from sagemaker.mxnet.model import MXNetModel
sagemaker_model = MXNetModel(model_data=model_data, role=role, entry_point='predict.py',
                             py_version='py3', framework_version='1.3.0')

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint. We can use the ``sagemaker.mxnet.model.MXNetModel.deploy`` method to do this. Unless you have created or prefer other instances, we recommend using 1 ``'ml.c4.xlarge'`` instance for this training. These are supplied as arguments. 

In [ ]:
import logging
logging.getLogger().setLevel(logging.WARNING)

predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Making an inference request

Now that our Endpoint is deployed and we have a ``predictor`` object which we can call for inference.

We wiill pass a single batch of an aligned image as a numpy array in the shape the model expects, setting `content_type` and `serializer` to convert into bytes.  The `predict.py` endpoint includes overides for `model_fn` to load fully connected layer and `transform_fn` to [transform](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html?highlight=input_fn#using-input-fn-predict-fn-and-output-fn) to load numpy input and return normalized emeddings as json.

The SageMaker MXNet containers are [open source](https://github.com/aws/sagemaker-containers) if you needed more details.

In [ ]:
!pygmentize predict.py

## Send requests

Download a sample picture, detect the first face

In [ ]:
import boto3
import json
import base64

s3 = boto3.resource('s3')

# Read image from s3
image = {
    'S3Object': {
        'Bucket': 'aiml-lab-sagemaker',
        'Name': 'politicians/politicians2.jpg'
    }
}
image_object = s3.Object(image['S3Object']['Bucket'] , image['S3Object']['Name'])
payload = image_object.get()['Body'].read()

rekognition = boto3.client('rekognition')
    
# Call rekognition to get bbox
ret = rekognition.detect_faces(
    Image={
        'Bytes': payload
    },
    Attributes=['DEFAULT'],
)

print(ret['FaceDetails'][0]['BoundingBox'])

Crop the image at the bounding box and resize for input into model

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

%matplotlib inline

def crop_image(payload, bbox, image_size=(112, 112)):
    # Load image and convert to RGB space
    img = cv2.imdecode(np.frombuffer(payload, np.uint8), cv2.IMREAD_COLOR) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Crop relative to image size
    if bbox != None:
        height, width, _ = img.shape
        x1 = int(bbox['Left'] * width)
        y1 = int(bbox['Top'] * height)
        x2 = int(bbox['Left'] * width + bbox['Width'] * width)
        y2 = int(bbox['Top'] * height + bbox['Height']  * height)
        img = img[y1:y2,x1:x2,:]
    # Resize
    return cv2.resize(img, (image_size[1], image_size[0]))    

image = crop_image(payload, ret['FaceDetails'][0]['BoundingBox'])
    
# Show the last image with size
plt.imshow(image)
image.shape

Transpose the image data into a numpy array the model expects and save file

In [ ]:
model_input = np.rollaxis(image, axis=2, start=0)[np.newaxis, :]
np.save('input.npy', model_input)
model_input.shape

Send the payload to the endpoint, and output the face embedding response

In [ ]:
%%time

def numpy_bytes_serializer(data):
    import io
    import numpy as np
    
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

model_input = np.load('input.npy')

In [ ]:
%%time

# Set the content-type to numpy 
predictor.accept = 'application/json'
predictor.content_type = 'application/x-npy'
predictor.serializer = numpy_bytes_serializer
response = predictor.predict(model_input)
predict_data = np.array(response)
print(np.array(response)[:10])

In [ ]:
%%time

# Call the predictor with just the image payload (offloading the rekognition bbox to endpoint)
predictor.accept = 'application/json'
predictor.content_type = 'application/x-image'
predictor.serializer = None
response = predictor.predict(payload)
predict_data = np.array(response)
print(np.array(response)[:10])

### Compare to Local

Extract the model locally and call `model_fn` from `predict.py` to load the model

In [ ]:
%%time

import os
import json
import numpy as np
from predict import model_fn, transform_fn

if not os.path.exists('model'):
    !mkdir -p model
    !tar xvzf model.tar.gz -C ./model    

model = model_fn('model')

Call `transform_fn` from `predict.py` to perform inference on test input

In [ ]:
%%time
        
model_input = np.load('input.npy')
data = numpy_bytes_serializer(model_input)
embedding, content_type = transform_fn(model, data, 'application/x-npy', 'application/json')
local_data = np.array(json.loads(embedding))
print(local_data[:10])

In [ ]:
(predict_data - local_data).sum()

## Clean up

Tear down the sagemaker endpoint

In [ ]:
predictor.delete_endpoint()

In [ ]:
import shutil

os.remove('input.npy')
os.remove('model.tar.gz')
shutil.rmtree('model')